In [147]:
using LinearAlgebra
#APOIOS

#
# Aplica condições de contorno essenciais 
# homogêneas
#
function Aplica_CC_homo!(apoios,K,F)

    # Aplica as condições de contorno essenciais homogêneas
    for i=1:size(apoios,1) 

        # No e gl local do apoio
        no  = Int(apoios[i,1])
        gll = Int(apoios[i,2])

        # Testa se o usuário tem ideia do que
        # ele está fazendo
        if apoios[i,3]!=0 
            error("Sua anta")
        end

        # Gl global
        gl = 2*(no-1) + gll

        # Zera linha e coluna da rigidez
        K[gl,:] .= 0.0
        K[:,gl] .= 0.0

        # Zera o vetor de forças nesse gl
        F[gl] = 0.0

        #Zera o vetor G nesse gl
        G[gl,:] .= 0.0

        # Coloca 1.0 na diagonal
        K[gl,gl] = 1.0

    end #gls

end

Aplica_CC_homo! (generic function with 2 methods)

In [163]:
#GLOBAL 

#
# conectividades é uma matriz com ne linhas (número de elementos)
# e 2 colunas (nó inicial e nó final)
#
function Global(dim,ne,nnos,conectividades,coord, VE,Vnuxy, esp)

    # Precisamos definir a matriz global
    K = zeros(dim,dim)

    # Aloca vetores para as coordenadas de cada elemento
    X = zeros(nnos)
    Y = zeros(nnos)
    nos = zeros(nnos)
    gls = zeros(Int,dim)
    
    # Loop nos elementos da malha, so tenho um elemento nesse primeiro problema.
    ele = 1

    # Recupera as informações do elemento
    Ee = VE[ele]
    ve = Vnuxy[ele]

    # Monta a matriz constitutiva do elemento
    C = Calcula_C(Ee,ve,"INC") #alterei o C para D com a hipotese de incompressibilidade.

    # Recupera as coordenadas do elemento
    for i=1:nnos
        # Descobre o nó
        no = conectividades[ele,i]
        
        # Guarda os nós do elemento
        nos[i] = no

        # Descobre as coordenadas
        X[i],Y[i] = coord[no,:]
    end
    
    # Monta a matriz de rigidez do elemento
    # no sistema local, com o elementos escolhido
    
    Kg = K_up(dim,C,X,Y,esp)

    return Kg
end

#
# Monta o vetor de força global
#
function Forca_global(nnos,forcas)

    # Montar o vetor de forças global
    F = zeros(2*nnos)

    # Para cada informação em forças 
    # posiciona a força no vetor de forças
    # globais do problema
    for i=1:size(forcas,1)
       
        # Recupera nó e gl local 
        no  = Int(forcas[i,1])
        gll = Int(forcas[i,2])

        # Recupera valor
        valor = forcas[i,3]

        # Adiciona ao valor da força
        F[2*(no-1)+gll] += valor

    end #i

    return F
end

function Gglobal(dim,ne,nnos,conectividades,coord, VE,Vnuxy, esp)
    # Aloca vetores para as coordenadas de cada elemento
    X = zeros(nnos)
    Y = zeros(nnos)
    nos = zeros(nnos)
    gls = zeros(Int,dim)

    # Loop nos elementos da malha
    ele=1

    # Recupera as informações do elemento
    Ee = VE[ele]
    ve = Vnuxy[ele]

    # Recupera as coordenadas do elemento
    for i=1:nnos
        # Descobre o nó
        no = conectividades[ele,i]
        
        # Guarda os nós do elemento
        nos[i] = no

        # Descobre as coordenadas
        X[i],Y[i] = coord[no,:]
    end

    # Monta a matriz G
    
    G = G_up(dim,X,Y,esp)
    M = M_up(dim,X,Y,Ee,ve)

    return G , M
end


Gglobal (generic function with 1 method)

In [164]:
#MATERIAL 

# Monta a matriz constitutiva dado E,v e a hipótese
function Calcula_C(E,v,hipotese="INC")

    # Testa se hipotese é válida
    hipotese in ["EPT","EPD", "INC"] || throw("Hipotese errada")
    
    G = E/(2*(1+v))

    # adicionando trecho incompressibilidade -
       
    if hipotese=="EPT"
        a = E/(1-v^2)
        C = [a v*a 0.0 ;
            v*a a  0.0 ;
              0  0  G]

    elseif hipotese=="EPD"
        d = (v+1)*(2*v-1)  
        a = E*(v-1)/d
        b = -(E*v)/d
        C = [a b 0 ;
             b a 0 ;
             0 0 G]
        
    elseif hipotese=="INC"
        μ = E/(2*(1+v))
        
        C = [2*μ 0    0 ;
              0   2*μ  0 ;
              0   0    μ ]
        
    else
 
        C = [a b b 0 0 0 ;
             b a b 0 0 0 ;
             b b a 0 0 0 ;
             0 0 0 G 0 0 ;
             0 0 0 0 G 0 ;
             0 0 0 0 0 G]

    end
    
    return C

end


Calcula_C (generic function with 2 methods)

In [266]:
#MATERIAL

# N1 = 0.25(1-r)(1-s)
# N2 = 0.25(1+r)(1-s)
# N3 = 0.25(1+r)(1+s)
# N4 = 0.25(1-r)(1+s)

#
# Calcula a matriz B e o determinante da matriz
# Jacobiana no ponto (r,s) 
#

#B para o procedimento no elemento u-p é o mesmo que dos elementos normaais.
function B_up(dim,r,s,X,Y)

   # Vetores com as derivadas das funções de interpolação
   # em relação a r e a s
   dNr = [ -0.25*(1-s) ; 0.25*(1-s) ; 0.25*(1+s)  ; -0.25*(1+s)   ]
   dNs = [ -0.25*(1-r) ;-0.25*(1+r) ; 0.25*(1+r)  ;  0.25*(1-r)   ]

   # Aloca e calcula a matriz J
   J = zeros(2,2)

   # Loop do somatório para cada posição de J
   for i=1:4
       J[1,1] += dNr[i]*X[i]
       J[1,2] += dNr[i]*Y[i]
       J[2,1] += dNs[i]*X[i]
       J[2,2] += dNs[i]*Y[i]
   end #i

   # Calcula o determinante
   dJ = det(J)

   # Calcula a inversa da J
   invJ = inv(J)

   # Aloca a matriz B
   B = zeros(3,dim)

   # Loop pelos blocos da B, com as correções de derivadas
   c = 1
   for i=1:4
     
       # Correção das derivadas para este bloco
       dNxy = invJ*[dNr[i];dNs[i]]

       # Posiciona na B
       B[1,c]   = dNxy[1]
       B[2,c+1] = dNxy[2]
       B[3,c]   = dNxy[2]
       B[3,c+1] = dNxy[1]

       # Atualiza a apontador de bloco
       c += 2

   end #i

   return dJ, B
end

#
# Monta a matriz de rigidez de um elemento
# bilinear isoparamétrico de 4 nós para formulação u-p. 
#
function K_up(dim, C::AbstractMatrix,X::Vector,Y::Vector,esp::Float64)

    # Aloca a matriz do elemento
    K = zeros(dim,dim)

    # Define a quadratura 
    pg = [-1/sqrt(3) ; 1/sqrt(3)]
    W  = [1.0 ; 1.0]

    # Loop pelos pontos de Gauss
    for i=1:2
        r  = pg[i]
        wr = W[i]
        for j=1:2
           s  = pg[j]  
           ws = W[j]

           # calcula o dJ e o B
           dJ, B = B_up(dim,r,s,X,Y)

           # Acumula o produto nesses ptos
           K = K + B'*C*B*dJ*esp

        end #j
    end # i

    # Retorna a matriz de rigidez do elmemento
    return K

end

#operador diferencial divergente

function Bb_up(dim,r,s,X,Y) 

   # Vetores com as derivadas das funções de interpolação
   # em relação a r e a s
   dNr = [ -0.25*(1-s) ; 0.25*(1-s) ; 0.25*(1+s)  ; -0.25*(1+s)   ]
   dNs = [ -0.25*(1-r) ;-0.25*(1+r) ; 0.25*(1+r)  ;  0.25*(1-r)   ]

   # Aloca e calcula a matriz J
   J = zeros(2,2)

   # Loop do somatório para cada posição de J
   for i=1:4
       J[1,1] += dNr[i]*X[i]
       J[1,2] += dNr[i]*Y[i]
       J[2,1] += dNs[i]*X[i]
       J[2,2] += dNs[i]*Y[i]
   end #i

   # Calcula o determinante
   dJ = det(J)

   # Calcula a inversa da J
   invJ = inv(J)

   # Aloca a matriz Bbolinha
   B = zeros(dim,1)

   # Loop pelos blocos da B, com as correções de derivadas
   c = 1

    #essa e a matriz Bbolinha que aplica o operador gradiente (∇)
   for i=1:4
        # Correção das derivadas para este bloco
        dNxy = invJ*[dNr[i];dNs[i]]
        
        # Posiciona na B
        B[c,1] = dNxy[1]
        B[c+1,1] = dNxy[2]
        
        # Atualiza a apontador de bloco
        c += 2

   end #i
   return dJ, B
end

#
# Monta a matriz G

function G_up(dim,X::Vector,Y::Vector,esp::Float64)

    # Aloca a matriz do elemento
    G = zeros(dim,1)

    # Define a quadratura 
    pg = [-1/sqrt(3) ; 1/sqrt(3)]
    W  = [1.0 ; 1.0]
    # Loop pelos pontos de Gauss
    for i=1:2
        r  = pg[i]
        wr = W[i]
        for j=1:2
            s  = pg[j]  
            ws = W[j]
            # calcula o dJ e o B
            dJ, B = Bb_up(dim,r,s,X,Y)

            
            # Acumula o produto nesses ptos
            G = G + B*dJ
        end #j
    end # i

    # Retorna a matriz de rigidez do elmemento
    return G

end

function M_up(dim,X::Vector,Y::Vector,E,ν)

    # Aloca a matriz do elemento
    M = 0
    #K = E / (3 * (1 - 2 * ν))
    K = (ν*E) / ((1+ν)*(1-2*ν))
    # Define a quadratura 
    pg = [-1/sqrt(3) ; 1/sqrt(3)]
    W  = [1.0 ; 1.0]
    # Loop pelos pontos de Gauss
    for i=1:2
        r  = pg[i]
        wr = W[i]
        for j=1:2
            s  = pg[j]  
            ws = W[j]
            # calcula o dJ e o B
            dJ, B = Bb_up(dim,r,s,X,Y)

            
            # Acumula o produto nesses ptos
            M = (M + 1/K)*dJ
        end #j
    end # i

    # Retorna a matriz de rigidez do elmemento
    return M

end


M_up (generic function with 2 methods)

In [267]:
#função que define o meu problema. Elemento com dimensoes diferentes. 
function incomp(v0)
    
    nnos = 4
    ne = 1
    
    #quantos graus de liberdade tenho por no:
    glsno = 2

    #variavel auxiliar que determina as dimensões do problema. N total de graus de liberdade
    dim = nnos*glsno 

    coord = [0.0   0.0   ; #n1
             2.0   0.0   ; #n2
             2.0   2.0   ; #n3
             0.0   2.0   ] #n4                      

    
    conectividades = [1 2 3 4]

    VE =    ones(ne) #Modulo de elasticidade do elemento
    Vnuxy = v0*ones(ne) #Poisson do elemento.
    
    E = VE[1]
    ν = Vnuxy[1]

    #Vou precisar para montar a matriz M.
    λ = (ν*E) / ((1+ν)*(1-2*ν))

    # Espessura
    esp = 1.0

    # Apoios (cond. de contorno essenciais)
    #        no gl valor
    apoios = [1 1 0.0;
              1 2 0.0;
              2 2 0.0]

    # Forças (cond. de contorno naturais)
    #         no gl valor
    P = 1
    forcas = [3  2  P;
              4  2  P]
    

    return nnos, ne,dim,coord,conectividades,VE,Vnuxy,esp,apoios,forcas,λ
end


#função que roda todas as rotinas, estou dando de entrada a função que define o meu problema e o poisson, pra ficar facil de testar.
function main_inc(func::Function, v0)

    nnos, ne,dim,coord,conectividades,VE,Vnuxy,esp,apoios,forcas,λ = func(v0)
    
    K = Global(dim,ne,nnos,conectividades,coord, VE, Vnuxy, esp)
    
    G,M = Gglobal(dim,ne,nnos,conectividades,coord, VE, Vnuxy, esp)
    #@show Mt
    
    F = Forca_global(nnos,forcas)

    # Modifica o sistema pela aplicação das condições de contorno
    # homogêneas
    Aplica_CC_homo!(apoios,K,F,G)

    
    #monta a matriz M:
    #M = (1/λ)*Mt
    #ones(1,1)

    #Matriz H zerada:
    H = zeros(1,1)

    
    #A = [K  G;
    #     G' M ]
    
    if Vnuxy[1]==0.5
        println("Caso incompressível, procedimento 2 Hughes: ")
        M = zeros(1,1)

        #G'INV(K)G)p = G'INV(K)F-H
        #Estou considerando H como 0.
        #G parece perfeitamente ok. 
        
        leq = G'*inv(K)*G
        req = G'*inv(K)*F 

        p = leq\req
        #Kd + Gp = F
        #Kd = F - Gp 

        req2 = F - G*p
        d = K\req2

    else
        #(K-G*inv(M)*G')*d = F-G*inv(M)*H  # 4.3.25
        #leq = (K-G*inv(M)*G')
        #req = F-GM*H = F
        
        println("Caso compressível, procedimento 1 Hughes: ")
        
        leq = K+(G*(G'*(1/M)))
        
        #req = F - (G*inv(M)*H) como H = 0 req = F
        req = F - (G*(1/M)*H)
        d = leq\req
        p = (1/M)*G'*d
        @show M
    end


    return d,p,K,G,M,λ
end




main_inc (generic function with 1 method)

In [268]:
using NBInclude
@nbinclude("casos.ipynb")

caso_d (generic function with 1 method)

In [269]:
d,p,K,G,M,λ = main_inc(caso_0,0.2)
display(d)
display(p)

Caso compressível, procedimento 1 Hughes: 
M = 14.399999999999999


8×1 Matrix{Float64}:
  0.0
  0.0
 -0.4800000000000004
  0.0
 -0.4800000000000009
  1.920000000000001
 -9.135549459772722e-16
  1.92

1×1 Matrix{Float64}:
 0.20000000000000004

In [270]:
t = 1/λ*ones(1,1)

1×1 Matrix{Float64}:
 3.5999999999999996

In [275]:
K[1,:]

8-element Vector{Float64}:
 1.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [260]:
p2

1×1 Matrix{Float64}:
 0.13513513513513517

In [157]:
d,p = main_inc(caso_B,0.499999999999)
display(d)
display(p)

Mt = [16.0;;]
Caso compressível, procedimento 1 Hughes: 


8×1 Matrix{Float64}:
 -1.20000187499333
  0.0
 -1.2000018749933299
  0.0
  0.0
  0.0
  0.0
  0.0

1×1 Matrix{Float64}:
 -0.011565079012604396

In [158]:
d,p = main_inc(caso_B,0.5)

Mt = [16.0;;]
Caso incompressível, procedimento 2 Hughes: 


([-1.2000000000000002, 0.0, -1.2, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0], [1666.6666666666665 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0 0.0 … 1.0 0.0; 0.0 0.0 … 0.0 1.0], [-1.0; 0.0; … ; 0.0; 0.0;;], [0.0;;], Inf)

In [159]:
p

1-element Vector{Float64}:
 0.0